In [37]:
# Importando Librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import SimpleImputer

#### Configurando Datos

In [38]:
# Cargando el Dataset
dataset = pd.read_csv('Data.csv')

In [39]:
# Mostrando una fracción del dataset
dataset.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


In [40]:
# Obteniendo datos de X
X = dataset.drop('Purchased',axis=1)
X

,Country,Age,Salary
0,France,44.0,72000.0
1,Spain,27.0,48000.0
2,Germany,30.0,54000.0
3,Spain,38.0,61000.0
4,Germany,40.0,NaN
5,France,35.0,58000.0
6,Spain,NaN,52000.0
7,France,48.0,79000.0
8,Germany,50.0,83000.0
9,France,37.0,67000.0


In [41]:
# Obteniendo datos de Y
y = dataset.pop('Purchased')
y

0     No
1    Yes
2     No
3     No
4    Yes
5    Yes
6     No
7    Yes
8     No
9    Yes
Name: Purchased, dtype: object

In [ ]:
# Mostrando dimensiones de X e y

print('X',X.shape)
print('y',y.shape)

In [42]:
# Tratando los datos NaN

# Estructurando el imputer para que reemplace los NaN por la media de cada columna 
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Procesando el conjunto de X
imputer = imputer.fit(X.iloc[:, 1:3])
X.iloc[:, 1:3] = imputer.transform(X.iloc[:,1:3])

In [ ]:
# Mostrando el conjunto de X con sus NaN reemplazados 
X

#### Codificar Datos Categoricos

In [43]:
from sklearn.preprocessing import LabelEncoder
labelencorer = LabelEncoder()
X.iloc[:,0] = labelencorer.fit_transform(X.iloc[:,0])

In [ ]:
# Mostrando la transformación categorica
X

<p style='text-align: justify;'>
La razón de transformar estos datos <b>categoricos ('Strings')</b> a datos <b>numericos ('Integer', 'Float')</b>, es por que a la hora de entrenar un conjunto de datos, nuestra red neuronal no puede procesar datos tipo string de forma sencilla. Es por esta razón por la que se decide realizar esta transformación con ayuda de Sklearn. Por lo que los datos <b>['France', 'Spain', 'Germany']</b>, pasan a ser <b>[0, 2, 1]</b><br>
</p>
<p style='text-align: justify;'>
Esto es una buena solución para poder procesar los datos a travez de una red neuronal. Sin embargo esta transformación daría a entender a la red neuronal que los datos tienen una relación de <b>[menor, medio y mayor]</b>, y como pueden ver esta representación aún que ya no es en string sigue siendo erronea por que los países no tienen correlaciones entre sí, como las prendas de ropa que tienen medidas <b>(CH, M, L)</b>, en este caso si tienen una relación de medidas para las prendas de ropa y en este caso si sería correcto esta solución.<br>
</p>
<p style='text-align: justify;'>
Por lo tanto para tener una solución mucho más correcta utilizaremos unos tipos de datos especiales, llamados <b>Dummy</b>
</p>

In [44]:
# Transfomrmación a Variables Dummy
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('one_hot_encoder', OneHotEncoder(categories='auto'),[0])],remainder='passthrough')

X = np.array(ct.fit_transform(X), dtype=np.int64)

In [21]:
X

array([[    1,     0,     0,    44, 72000],
       [    0,     0,     1,    27, 48000],
       [    0,     1,     0,    30, 54000],
       [    0,     0,     1,    38, 61000],
       [    0,     1,     0,    40, 63777],
       [    1,     0,     0,    35, 58000],
       [    0,     0,     1,    38, 52000],
       [    1,     0,     0,    48, 79000],
       [    0,     1,     0,    50, 83000],
       [    1,     0,     0,    37, 67000]], dtype=int64)

<p style='text-align: justify;'>
Como pueden ver en la impresión anterior separamos la lista de paises en 3 columnas, una por cada tipo de categoría que se encontró en esa columna, dichas columnas solo tendrán un <b>1</b> para especificar que en esa fila se encuentra dicha categoría, por otro lado, se pondrá un <b>0</b> para representar que en esa fila no se encuentra dicha categoría 
</p>

In [45]:
# Transformando valores de Y

y = labelencorer.fit_transform(y)

In [33]:
y

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

#### Dividir el dataset en train y test

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state = 0)


In [35]:
print('X train:',X_train)
print('----------------------')
print('X test',X_test)
print('----------------------')
print('y train',y_train)
print('----------------------')
print('y test',y_test)
print('----------------------')

X train: [[    0     1     0    40 63777]
 [    1     0     0    37 67000]
 [    0     0     1    27 48000]
 [    0     0     1    38 52000]
 [    1     0     0    48 79000]
 [    0     0     1    38 61000]
 [    1     0     0    44 72000]
 [    1     0     0    35 58000]]
----------------------
X test [[    0     1     0    30 54000]
 [    0     1     0    50 83000]]
----------------------
y train [1 1 1 0 1 0 0 1]
----------------------
y test [0 0]
----------------------


#### Escalado de datos

<p style='text-align: justify;'>
En este apartado se utilizará una tecnica para poder escalar los datos, el escalado de datos es utilizado para poder establecer un margen de valores para todas las columnas que se vallan a utilizar en el dataframe de entrenamiento y testing.<br>
</p>

<p style='text-align: justify;'>
Este metodo es utilizado solamente para casos muy especificos en los que las columnas de datos que queremos procesar tienen una discrepancia muy grande entre ellas, en este caso siendo <b>['Age', 'Salary']</b>, estos datos son muy importantes para ser usados en nuestro conjunto de datos, sin embargo estos datos tienen magnitudes muy diferentes entre sí, por lo que la computadora podría tener problemas para relacionar unas con la sotras, ya que el salario se muestre en miles de unidades y los años en tansolo decenas.<br>
</p>
<p style='text-align: justify;'>
Por esta razón se escalarán los datos, y para esta tarea contamos con dos formas diferentes de escalar los datos. Podemos <b>Normalizar</b> los datos, lo que significa que debemos distribuir los valores de una columna entre el -1 y el 1, Esta es una buena forma de escalado para los datos, aún que en esta ocación utilizaremos la <b>Estandarización</b> de datos, siendo esta como su nombre lo dice, sacar la desviación estandar en los datos y utilizarlo para el escalado de los datos, esto en su mayoría puede ser mucho mejor que simplemente convertir los datos de -1 a 1, ya que los datos son escalados de tal forma que preservar cierta escencia de los datos originales.<br>
</p>



In [49]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [50]:
# Mostrando datos Estandarizados
print('X train:',X_train)
print('---------------------------------------')
print('X test:',X_test)

X train: [[-1.          2.64575131 -0.77459667  0.27978024  0.12374357]
 [ 1.         -0.37796447 -0.77459667 -0.23673712  0.4617671 ]
 [-1.         -0.37796447  1.29099445 -1.95846165 -1.53092514]
 [-1.         -0.37796447  1.29099445 -0.06456467 -1.11141099]
 [ 1.         -0.37796447 -0.77459667  1.65715986  1.72030956]
 [-1.         -0.37796447  1.29099445 -0.06456467 -0.16750414]
 [ 1.         -0.37796447 -0.77459667  0.96847005  0.98615979]
 [ 1.         -0.37796447 -0.77459667 -0.58108203 -0.48213975]]
---------------------------------------
X test: [[-1.          2.64575131 -0.77459667 -1.44194429 -0.90165391]
 [-1.          2.64575131 -0.77459667  2.00150476  2.13982372]]
